# VieNeu-TTS API on Google Colab with Ngrok

Notebook này giúp bạn chạy VieNeu-TTS API trên Google Colab và expose ra ngoài qua Ngrok.

## Các bước thực hiện:
1. Clone repository
2. Cài đặt dependencies
3. Setup Ngrok
4. Khởi động API server
5. Test API

## 1. Kiểm tra GPU

In [ ]:
!nvidia-smi

## 2. Clone Repository

In [ ]:
# Clone repository (thay YOUR_REPO_URL bằng URL repo của bạn)
!git clone YOUR_REPO_URL vieneu-tts
%cd vieneu-tts

## 3. Cài đặt Dependencies

In [ ]:
# Cài đặt requirements
!pip install -q -r requirements.txt

# Cài đặt thêm uvicorn và pyngrok
!pip install -q uvicorn pyngrok

## 4. Setup Ngrok

Bạn cần có Ngrok auth token. Lấy token tại: https://dashboard.ngrok.com/get-started/your-authtoken

In [ ]:
from pyngrok import ngrok, conf
import os

# Nhập Ngrok auth token của bạn
NGROK_AUTH_TOKEN = "YOUR_NGROK_AUTH_TOKEN"  # Thay bằng token của bạn

# Set auth token
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

print("✅ Ngrok configured successfully!")

## 5. Khởi động API Server với Ngrok

In [ ]:
import threading
import uvicorn
from pyngrok import ngrok
import time

# Import API app
from api_server import app

# Port cho API
PORT = 8000

# Khởi động ngrok tunnel
public_url = ngrok.connect(PORT)
print(f"\n🌐 Public URL: {public_url}")
print(f"📖 API Docs: {public_url}/docs")
print(f"\n⚠️ Lưu URL này để gọi API từ bên ngoài!\n")

# Khởi động FastAPI server trong thread riêng
def run_server():
    uvicorn.run(app, host="0.0.0.0", port=PORT, log_level="info")

server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()

print("\n🚀 Server is starting...")
time.sleep(5)
print("✅ Server is ready!")

## 6. Load Model

Trước khi sử dụng API, bạn cần load model trước.

In [ ]:
import requests
import json

# Lấy public URL (từ cell trước)
api_url = str(public_url)

# Load model
load_config = {
    "backbone": "VieNeu-TTS (GPU)",  # Hoặc "VieNeu-TTS-q4-gguf" cho CPU
    "codec": "NeuCodec (Standard)",
    "device": "Auto",
    "enable_triton": True,
    "max_batch_size": 8
}

print("⏳ Loading model...")
response = requests.post(f"{api_url}/load_model", json=load_config)

if response.status_code == 200:
    result = response.json()
    print("\n✅ Model loaded successfully!")
    print(json.dumps(result, indent=2, ensure_ascii=False))
else:
    print(f"\n❌ Error: {response.status_code}")
    print(response.text)

## 7. Test API - Synthesize Speech

In [ ]:
import requests
from IPython.display import Audio

# Test synthesize
tts_request = {
    "text": "Xin chào, đây là hệ thống chuyển đổi văn bản thành giọng nói tiếng Việt.",
    "voice": "Vĩnh (nam miền Nam)",
    "use_batch": True
}

print("⏳ Synthesizing speech...")
response = requests.post(f"{api_url}/synthesize", json=tts_request)

if response.status_code == 200:
    # Save audio file
    with open("output.wav", "wb") as f:
        f.write(response.content)
    
    print("\n✅ Speech synthesized successfully!")
    print("🔊 Playing audio...")
    
    # Play audio in notebook
    display(Audio("output.wav", autoplay=True))
else:
    print(f"\n❌ Error: {response.status_code}")
    print(response.text)

## 8. Test API - Get Base64 Audio

In [ ]:
import requests
import json
import base64
from IPython.display import Audio

# Test synthesize with base64 response
tts_request = {
    "text": "Đây là test trả về audio dưới dạng base64.",
    "voice": "Ngọc (nữ miền Bắc)",
    "use_batch": True
}

print("⏳ Synthesizing speech (base64)...")
response = requests.post(f"{api_url}/synthesize_base64", json=tts_request)

if response.status_code == 200:
    result = response.json()
    print("\n✅ Speech synthesized successfully!")
    print(f"Duration: {result['duration']:.2f}s")
    print(f"Sample Rate: {result['sample_rate']}Hz")
    
    # Decode base64 and save
    audio_bytes = base64.b64decode(result['audio_base64'])
    with open("output_base64.wav", "wb") as f:
        f.write(audio_bytes)
    
    print("\n🔊 Playing audio...")
    display(Audio("output_base64.wav", autoplay=True))
else:
    print(f"\n❌ Error: {response.status_code}")
    print(response.text)

## 9. Check Status

In [ ]:
import requests
import json

# Check status
response = requests.get(f"{api_url}/status")

if response.status_code == 200:
    status = response.json()
    print("📊 Server Status:")
    print(json.dumps(status, indent=2, ensure_ascii=False))
else:
    print(f"❌ Error: {response.status_code}")

## 10. List Available Voices

In [ ]:
import requests
import json

# List voices
response = requests.get(f"{api_url}/voices")

if response.status_code == 200:
    voices = response.json()
    print("🎤 Available Voices:")
    print(json.dumps(voices, indent=2, ensure_ascii=False))
else:
    print(f"❌ Error: {response.status_code}")

## 11. Keep Server Running

Chạy cell này để giữ server hoạt động. Nhấn Stop để dừng.

In [ ]:
import time

print(f"\n🌐 Public URL: {public_url}")
print(f"📖 API Docs: {public_url}/docs")
print("\n⚠️ Server đang chạy. Nhấn Stop button để dừng.\n")

try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("\n🛑 Server stopped.")
    ngrok.disconnect(public_url)

## Ví dụ gọi API từ Python (bên ngoài Colab)

```python
import requests

# Thay YOUR_NGROK_URL bằng URL từ Colab
API_URL = "https://xxxx-xx-xxx-xxx-xxx.ngrok-free.app"

# 1. Load model
load_config = {
    "backbone": "VieNeu-TTS (GPU)",
    "codec": "NeuCodec (Standard)",
    "device": "Auto",
    "enable_triton": True,
    "max_batch_size": 8
}
response = requests.post(f"{API_URL}/load_model", json=load_config)
print(response.json())

# 2. Synthesize speech
tts_request = {
    "text": "Xin chào từ hệ thống bên ngoài!",
    "voice": "Vĩnh (nam miền Nam)",
    "use_batch": True
}
response = requests.post(f"{API_URL}/synthesize", json=tts_request)

# Save audio
with open("output.wav", "wb") as f:
    f.write(response.content)
print("✅ Audio saved to output.wav")
```

## Ví dụ gọi API từ cURL

```bash
# Load model
curl -X POST "https://xxxx.ngrok-free.app/load_model" \
  -H "Content-Type: application/json" \
  -d '{
    "backbone": "VieNeu-TTS (GPU)",
    "codec": "NeuCodec (Standard)",
    "device": "Auto",
    "enable_triton": true,
    "max_batch_size": 8
  }'

# Synthesize speech
curl -X POST "https://xxxx.ngrok-free.app/synthesize" \
  -H "Content-Type: application/json" \
  -d '{
    "text": "Xin chào!",
    "voice": "Vĩnh (nam miền Nam)",
    "use_batch": true
  }' \
  --output output.wav
```